# Exploratory Data Analysis - House Prices

## Loading packages

In [15]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [16]:
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    # Kaggle-specific configuration
    TEST_PATH = Path('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
    TRAIN_PATH = Path('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
else:
    # Local configuration
    TEST_PATH = Path('data/test.csv')
    TRAIN_PATH = Path('data/train.csv')

In [17]:
house_prices_test_df = pd.read_csv(TEST_PATH)
house_prices_train_df = pd.read_csv(TRAIN_PATH)

## Dataset Info

In [21]:
print(house_prices_test_df.shape)
print(house_prices_train_df.shape)

(1459, 80)
(1460, 81)
